# Visão Geral do Processo
- A solução envolve três partes principais:

  - FastAPI (Python): Para servir o modelo de machine learning e fazer previsões de falência.
  - Backend em Node.js: Para interagir com a API FastAPI e servir o frontend.
  - Frontend em React: Para fornecer uma interface de usuário onde os dados podem ser inseridos e a previsão de falência pode ser exibida.
- API FastAPI para Servir o Modelo de Machine Learning
- O FastAPI será responsável por carregar o modelo treinado e expô-lo por meio de uma API RESTful. Aqui, usaremos o modelo de RandomForestClassifier treinado com SMOTE e ajustado por um limiar de decisão.

## Estrutura de Diretórios da API FastAPI:



In [ ]:
/fastapi-app
    /model
        model.pkl   (Modelo treinado)
    main.py        (Código FastAPI)
    requirements.txt


# Instalar Dependências:
- No arquivo requirements.txt, insira:




In [ ]:
fastapi
uvicorn
scikit-learn
imbalanced-learn
pandas
joblib


# Instale as dependências com o comando:

In [ ]:
pip install -r requirements.txt


# Código da API FastAPI (main.py):

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import pandas as pd

# Inicializando a aplicação FastAPI
app = FastAPI()

# Carregar o modelo treinado
model = joblib.load("model/model.pkl")

# Definir o esquema de entrada
class InputData(BaseModel):
    dependencia_emprestimos: float
    rendimento_liquido_patrimonio_acionistas: float
    divida_total_valor_liquido_total: float
    roa_b_antes_juros_depreciacao_apos_imposto: float
    receita_despesa_extra_industria: float
    indice_despesas_juros: float
    valor_liquido_ativos: float
    indice_endividamento: float
    caixa_ativos_totais: float
    capital_giro_patrimonio_liquido: float

# Função para predizer a falência com ajuste de limiar
@app.post("/predict/")
async def predict(data: InputData, threshold: float = 0.5):
    input_df = pd.DataFrame([data.dict()])

    # Realiza a previsão
    prediction_proba = model.predict_proba(input_df)[:, 1]  # Probabilidade da classe 1 (falência)
    prediction = (prediction_proba > threshold).astype(int)  # Ajuste do limiar de decisão

    return {"prediction": int(prediction[0]), "probability": float(prediction_proba[0])}

# Rodar o servidor com 'uvicorn main:app --reload' no terminal


# Como funciona o código:
- O modelo treinado é carregado usando joblib.load.
- O endpoint /predict/ recebe os dados do usuário e calcula a probabilidade de falência.
- O limiar de decisão é ajustável, com um valor padrão de 0.5. O parâmetro threshold pode ser passado para alterar esse limiar (ex: ?threshold=0.7).
- A resposta inclui a classe prevista e a probabilidade associada.
## Executar o servidor FastAPI:

In [ ]:
uvicorn main:app --reload


- A API estará disponível em http://127.0.0.1:8000.

# Backend em Node.js
- Agora, vamos configurar o backend em Node.js para fazer as requisições à API FastAPI e servir os dados para o frontend.

### Estrutura de Diretórios do Backend:

In [ ]:
/node-backend
    server.js
    package.json


# Código do Backend (server.js):
- Instalar dependências:
  - Primeiro, crie o arquivo package.json e instale as dependências necessárias (express para o servidor e axios para fazer as requisições HTTP):

In [ ]:
npm init -y
npm install express axios cors


# Código do servidor Node.js (server.js):

In [ ]:
const express = require("express");
const axios = require("axios");
const cors = require('cors');

const app = express();
const port = 3000;

// Habilitar CORS para permitir requisições do frontend
app.use(cors());
app.use(express.json());

// Rota para consumir a API FastAPI e obter a previsão
app.post("/predict", async (req, res) => {
    try {
        const response = await axios.post("http://127.0.0.1:8000/predict/", req.body);
        res.json(response.data);
    } catch (error) {
        console.error("Erro ao fazer a requisição para a API FastAPI:", error.message);
        res.status(500).json({ error: "Erro ao fazer a previsão" });
    }
});

// Iniciar o servidor
app.listen(port, () => {
    console.log(`Servidor Node.js rodando na porta ${port}`);
});


# Explicação:
- O servidor Node.js usa Express para gerenciar as rotas e Axios para fazer requisições HTTP para a API FastAPI.
- A rota /predict recebe os dados do frontend, envia para a API FastAPI e retorna a resposta (previsão de falência e a probabilidade).
CORS é habilitado para permitir que o frontend React acesse a API.
### Executar o servidor Node.js:

In [ ]:
node server.js


- O backend estará rodando em http://127.0.0.1:3000.

# Frontend em React
- Agora, vamos criar o frontend em React, onde o usuário poderá inserir as variáveis e ver o resultado da previsão de falência.

### Estrutura de Diretórios do Frontend:

In [ ]:
/frontend
    /src
        App.js
    package.json


# Código do Frontend (App.js):
- Instalar o React:
Se você ainda não tem um projeto React, crie um com o seguinte comando:




In [ ]:
npx create-react-app frontend


# Código do React (App.js):

In [ ]:
import React, { useState } from "react";
import axios from "axios";

function App() {
  const [inputData, setInputData] = useState({
    dependencia_emprestimos: "",
    rendimento_liquido_patrimonio_acionistas: "",
    divida_total_valor_liquido_total: "",
    roa_b_antes_juros_depreciacao_apos_imposto: "",
    receita_despesa_extra_industria: "",
    indice_despesas_juros: "",
    valor_liquido_ativos: "",
    indice_endividamento: "",
    caixa_ativos_totais: "",
    capital_giro_patrimonio_liquido: ""
  });

  const [prediction, setPrediction] = useState(null);
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState(null);

  const handleChange = (e) => {
    const { name, value } = e.target;
    setInputData({
      ...inputData,
      [name]: value
    });
  };

  const handleSubmit = async (e) => {
    e.preventDefault();

    // Validação simples
    if (Object.values(inputData).some(value => value === "")) {
      setError("Todos os campos precisam ser preenchidos.");
      return;
    }

    setLoading(true);
    setError(null);

    try {
      const response = await axios.post("http://127.0.0.1:3000/predict", inputData);
      setPrediction(response.data);
    } catch (error) {
      setError("Erro ao fazer a requisição para o backend.");
    } finally {
      setLoading(false);
    }
  };

  return (
    <div>
      <h1>Previsão de Falência</h1>
      <form onSubmit={handleSubmit}>
        {Object.keys(inputData).map((key) => (
          <div key={key}>
            <label>{key.replace(/_/g, " ")}</label>
            <input
              type="number"
              name={key}
              value={inputData[key]}
              onChange={handleChange}
              required
            />
          </div>
        ))}
        <button type="submit">Enviar</button>
      </form>

      {loading && <p>Carregando...</p>}
      {error && <p style={{ color: 'red' }}>{error}</p>}

      {prediction && (
        <div>
          <h2>Resultado da Previsão</h2>
          <p>Previsão de Falência: {prediction.prediction ? "Sim" : "Não"}</p>
          <p>Probabilidade: {prediction.probability}</p>
        </div>
      )}
    </div>
  );
}

export default App;


# Explicação:
- O formulário coleta as 10 variáveis do usuário, que são enviadas para o backend Node.js.
- A requisição POST é feita para o backend Node.js, que por sua vez faz a requisição à API FastAPI.
- A resposta é exibida no frontend, mostrando a previsão de falência e a probabilidade.
### Executar o Frontend:
- Para rodar o frontend, execute o seguinte comando dentro do diretório do projeto React:

In [ ]:
npm start


- O frontend estará disponível em http://localhost:3000.

# Explicação sobre o Limiar (Threshold)
- O limiar é o valor de corte utilizado para decidir a classe de uma previsão com base na probabilidade gerada pelo modelo. Em uma classificação binária (como a previsão de falência), o modelo retorna uma probabilidade de a instância pertencer à classe "falência" (classe 1).
- O limiar define quando essa probabilidade é suficiente para classificar a instância como falida.

- Limiar padrão de 0.5: Se a probabilidade for maior que 50%, a empresa é classificada como falida (classe 1); caso contrário, ela é classificada como não falida (classe 0).
- Ajustando o limiar: O limiar pode ser ajustado para ser mais sensível ou mais conservador. Um limiar mais baixo (ex: 0.3) fará com que o modelo classifique mais empresas como falidas, aumentando a revocação (mas reduzindo a precisão).
- Já um limiar mais alto (ex: 0.7) será mais rigoroso e apenas classificará empresas com alta probabilidade de falir.
- Esse ajuste é importante para controlar a taxa de falsos positivos e falsos negativos com base na estratégia de negócio.

# Arquivo package.json para o Backend Node.js
- Quando você cria o projeto Node.js, você pode gerar o arquivo package.json automaticamente com o comando npm init -y. No entanto, para garantir que as dependências corretas estejam configuradas, o package.json para o backend Node.js que interage com a API FastAPI e serve o frontend React seria semelhante ao seguinte:

## Conteúdo do package.json:

In [ ]:
{
  "name": "node-backend",
  "version": "1.0.0",
  "description": "Backend para API de previsão de falência usando FastAPI e React",
  "main": "server.js",
  "scripts": {
    "start": "node server.js"
  },
  "author": "",
  "license": "ISC",
  "dependencies": {
    "axios": "^1.4.0",
    "cors": "^2.8.5",
    "express": "^4.18.2"
  }
}


# Explicação:
- name: Nome do seu projeto (node-backend).
- version: Versão do seu projeto.
- description: Descrição breve sobre o que é o seu projeto.
- main: Arquivo principal do seu servidor, que é o server.js no seu caso.
- scripts:
- "start": Comando para iniciar o servidor Node.js. Quando você rodar - npm start, ele vai iniciar o servidor com node server.js.
- dependencies:
- axios: Para fazer requisições HTTP para a API FastAPI.
- cors: Para permitir que o frontend (React) faça requisições para o backend (Node.js).
- express: Framework para criar o servidor e gerenciar as rotas.
### Como criar o arquivo package.json:
- Se você ainda não tem o arquivo package.json no seu projeto, você pode criar um novo arquivo com o seguinte comando:




In [ ]:
npm init -y


- Em seguida, instale as dependências necessárias com o comando:

In [ ]:
npm install axios cors express


- Isso vai gerar um arquivo package.json automaticamente, e as dependências serão adicionadas à seção dependencies.

#  package.json para o Backend (Node.js)
- Este é o arquivo package.json que você terá no backend, dentro do diretório node-backend, como mencionei antes. Ele contém as dependências necessárias para o servidor Node.js que vai se comunicar com a API FastAPI.

- Exemplo de package.json para o Backend (Node.js):

In [ ]:
{
  "name": "node-backend",
  "version": "1.0.0",
  "description": "Backend para API de previsão de falência usando FastAPI e React",
  "main": "server.js",
  "scripts": {
    "start": "node server.js"
  },
  "author": "",
  "license": "ISC",
  "dependencies": {
    "axios": "^1.4.0",
    "cors": "^2.8.5",
    "express": "^4.18.2"
  }
}


# Explicação:

- axios: Faz as requisições HTTP para a API FastAPI.
express: Framework para criar o servidor e gerenciar as rotas.
cors: Para permitir requisições do frontend (React) para o backend (Node.js).
# package.json para o Frontend (React)
- Já o frontend será feito com React. O arquivo package.json para o frontend será gerado automaticamente quando você criar o projeto React usando o create-react-app. Esse arquivo vai gerenciar as dependências do lado cliente.

## Exemplo de package.json para o Frontend (React):

In [ ]:
{
  "name": "frontend",
  "version": "0.1.0",
  "private": true,
  "dependencies": {
    "axios": "^1.4.0",
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "react-scripts": "5.0.0"
  },
  "scripts": {
    "start": "react-scripts start",
    "build": "react-scripts build",
    "test": "react-scripts test",
    "eject": "react-scripts eject"
  },
  "eslintConfig": {
    "extends": [
      "react-app",
      "react-app/jest"
    ]
  },
  "browserslist": {
    "production": [
      ">0.2%",
      "not dead",
      "not op_mini all"
    ],
    "development": [
      "last 1 chrome version",
      "last 1 firefox version",
      "last 1 safari version"
    ]
  }
}


# Explicação:

- axios: Usado no frontend para fazer requisições ao backend Node.js.
- react e react-dom: As dependências principais do React.
- react-scripts: Utilizado para iniciar o ambiente de desenvolvimento React.
- Quando você cria um projeto React com create-react-app, ele já gera automaticamente o arquivo package.json com essas dependências e scripts configurados. Por exemplo, você pode rodar npm start para iniciar o servidor de desenvolvimento do React.

## Estrutura do Projeto
- Aqui está como a estrutura do seu projeto deve ficar, com os dois diretórios (frontend e node-backend), cada um com seu respectivo package.json:

In [ ]:
/meu-projeto
    /frontend              <- Diretório do React (Frontend)
        /src
            App.js
        package.json        <- package.json para o React
    /node-backend           <- Diretório do Node.js (Backend)
        server.js
        package.json        <- package.json para o Node.js
    /fastapi-app            <- Diretório da API FastAPI
        main.py
        model/
            model.pkl
        requirements.txt


# Como Criar os Dois Projetos
- Frontend (React):

- No diretório frontend, você cria o projeto React com:

In [ ]:
npx create-react-app frontend


# Isso vai gerar a estrutura do React, incluindo o package.json para o frontend.

## Backend (Node.js):

- No diretório node-backend, você inicializa o projeto Node.js com:

In [ ]:
npm init -y


### E depois instala as dependências necessárias:

In [ ]:
npm install axios cors express


### Isso vai gerar o arquivo package.json para o backend.

# Resumo
- package.json do Backend (Node.js): Gerencia as dependências para o servidor Express (com Axios e CORS).
- package.json do Frontend (React): Gerencia as dependências do React (incluindo Axios para fazer requisições HTTP).
- Cada parte do projeto tem seu próprio arquivo package.json e as dependências apropriadas.
- Com essa estrutura, seu projeto estará bem organizado, e você poderá desenvolver o backend e o frontend de forma independente, mas comunicando-os por meio da API FastAPI e do servidor Node.js.

# Como utilizar e colunas para o formulario
1. Dependência de Empréstimos
- Pergunta no Formulário:

  - "Qual a porcentagem das suas dívidas totais que é financiada por empréstimos?"
  - Como obter esse valor:

  - "Para calcular esse valor, basta dividir o total de empréstimos que sua empresa tem (seja com bancos, financeiras ou outras fontes) pelo total das dívidas totais da empresa. Se você tem um extrato de dívidas ou um relatório de financiamentos, esse número estará lá. Depois, multiplique por 100 para ter o valor em porcentagem."
- Explicação para o usuário:
"Este valor mostra o quanto sua empresa depende de empréstimos para manter suas operações. Se for muito alto, isso pode ser um sinal de que sua empresa pode estar com dificuldades financeiras."
2. Rentabilidade sobre o Patrimônio dos Sócios
- Pergunta no Formulário:
  - "Quanto a sua empresa lucra em relação ao valor investido pelos sócios?"
  - Como obter esse valor:
  - "Primeiro, calcule o lucro líquido da sua empresa (o que sobra após todos os custos e impostos). Em seguida, divida esse lucro pelo valor total que os sócios investiram na empresa (o patrimônio deles). Multiplique o resultado por 100 para ter o valor em porcentagem."
- Explicação para o usuário:
  - "Este indicador mostra se a empresa está gerando lucro em relação ao dinheiro que os sócios colocaram nela. Se esse valor for muito baixo, pode ser que sua empresa não esteja sendo eficiente em gerar retorno."
3. Dívida Total vs. Valor Líquido da Empresa
- Pergunta no Formulário:
  - "Qual o valor total das suas dívidas em relação ao valor que sua empresa possui de ativos líquidos?"
  - Como obter esse valor:
  - "Para calcular isso, some todas as dívidas da empresa, incluindo empréstimos, financiamentos e qualquer outra obrigação financeira. Em seguida, verifique o valor de seus ativos líquidos (dinheiro e bens que podem ser rapidamente convertidos em dinheiro). Divida o total de dívidas pelo valor dos ativos líquidos e multiplique por 100 para obter a porcentagem."
- Explicação para o usuário:
"Esse valor mostra se a empresa tem mais dívidas do que o que possui em dinheiro e bens líquidos. Se a dívida for maior, isso pode indicar que a empresa está em risco de não conseguir pagar suas obrigações."
4. Retorno sobre os Ativos
- Pergunta no Formulário:
  - "Quanto sua empresa lucra em relação aos seus bens e propriedades?"
  - Como obter esse valor:
  - "Primeiro, calcule o lucro líquido da sua empresa (como já foi explicado). Em seguida, some o valor de todos os ativos da empresa (bens, imóveis, equipamentos, etc.). Divida o lucro líquido pelo total de ativos e multiplique por 100 para obter a porcentagem."
- Explicação para o usuário:
- "Esse número mostra o quanto sua empresa consegue gerar de lucro com os bens e propriedades que possui. Se o retorno for baixo, pode ser que a empresa não esteja aproveitando bem seus recursos."
5. Despesas Extraordinárias com a Indústria
- Pergunta no Formulário:

  - "Sua empresa tem gastos extras com a indústria? Se sim, qual o valor desses gastos?"
- Como obter esse valor:

  - "Esses gastos podem ser encontrados em seus relatórios financeiros, especificamente na parte que fala sobre despesas operacionais ou custos extraordinários. Se sua empresa teve custos não recorrentes (como manutenção de equipamentos ou aquisição de matérias-primas a um custo inesperado), some esses valores."
- Explicação para o usuário:

- "Esses gastos representam custos adicionais que sua empresa tem fora do normal. Se eles forem muito altos, podem afetar sua rentabilidade e saúde financeira."
6. Índice de Despesas com Juros
- Pergunta no Formulário:
  - "Qual é a porcentagem da sua receita que é usada para pagar juros de dívidas?"
- Como obter esse valor:
  - "Para calcular, verifique os pagamentos de juros feitos pela sua empresa (geralmente na seção de ‘despesas financeiras’ do seu balanço). Depois, divida o total de juros pagos pela sua receita anual. Multiplique o resultado por 100 para obter a porcentagem."

- Explicação para o usuário:

- "Se um grande valor da sua receita é destinado ao pagamento de juros, isso pode indicar que sua empresa está gastando mais para manter suas dívidas do que para crescer."
7. Valor Líquido vs. Ativos Totais
- Pergunta no Formulário:
  - "Qual a porcentagem do valor disponível da sua empresa em relação aos seus ativos totais?"
  - Como obter esse valor:
  - "Verifique o saldo de caixa da sua empresa (dinheiro disponível ou reservas financeiras). Em seguida, divida esse valor pelo total de ativos que a empresa possui (todos os bens e propriedades). Multiplique o resultado por 100 para ter o valor em porcentagem."
- Explicação para o usuário:
  - "Esse número mostra a proporção do que sua empresa tem de recursos líquidos comparado ao total de seus bens. Se for baixo, pode significar que a empresa não está bem preparada para imprevistos."
8. Índice de Endividamento
- Pergunta no Formulário:
  - "Qual a porcentagem do valor total da sua empresa é composta por dívidas?"
- Como obter esse valor:
  - "Para calcular, divida o total de dívidas da empresa pelo valor total da empresa (soma do patrimônio líquido mais as dívidas). Multiplique o resultado por 100 para obter a porcentagem."
- Explicação para o usuário:
  - "Esse valor ajuda a entender o quanto a empresa deve em comparação com o que ela possui. Quanto maior a porcentagem, mais a empresa está endividada."
9. Proporção de Caixa em Relação aos Ativos Totais
- Pergunta no Formulário:

  - "Qual a porcentagem do valor total da sua empresa está disponível em caixa ou reservas?"
- Como obter esse valor:

  - "Esse valor pode ser encontrado no balanço financeiro da empresa. Verifique o total de caixa e reservas e divida pelo total de ativos da empresa. Multiplique por 100 para obter a porcentagem."
- Explicação para o usuário: "Esse indicador mostra se a empresa tem dinheiro suficiente em caixa para cobrir suas obrigações imediatas. Quanto maior esse valor, mais fácil será para a empresa lidar com imprevistos."

10. Capital de Giro em Relação ao Patrimônio Líquido
- Pergunta no Formulário:

  - "Qual a proporção do seu capital de giro (dinheiro disponível para operar) em relação ao patrimônio dos sócios?"
- Como obter esse valor:

  - "O capital de giro pode ser calculado a partir da diferença entre o que a empresa possui em ativos circulantes (dinheiro, contas a receber, estoques) e as suas obrigações de curto prazo. Divida esse valor pelo total do patrimônio líquido dos sócios e multiplique por 100 para obter a porcentagem."
- Explicação para o usuário:

  - "O capital de giro é o dinheiro necessário para a empresa operar no dia a dia. Se o valor for muito baixo, pode indicar que a empresa não tem recursos suficientes para manter suas operações sem recorrer a crédito ou empréstimos."
### Dicas para o Formulário:

- Exemplos Visuais: Sempre que possível, forneça exemplos visuais ou valores padrão para facilitar a compreensão, como "Se você tem R100.000emdívidaseR 50.000 em recursos líquidos, a dependência de empréstimos seria de 200%."
- Campos Auxiliares: Considere incluir links para exemplos ou guias passo a passo, caso o usuário precise de ajuda para encontrar ou calcular algum valor.
- Uso de Escalas e Barras Deslizantes: Para valores em porcentagens, você pode usar barras deslizantes (sliders), onde o usuário apenas escolhe um valor entre 0% e 100%, simplificando o processo de input.
- Como o Modelo de IA usa os dados:

- Esses valores serão usados por um modelo de IA para entender melhor a situação financeira da empresa. Com base nas entradas que o usuário fornecer, a IA pode identificar padrões e sinais que indicam risco de falência, levando em consideração a combinação desses fatores, como endividamento excessivo, baixo retorno sobre ativos e falta de liquidez.
- Esse processo pode envolver algoritmos como análise preditiva ou aprendizado supervisionado, onde o modelo foi treinado em dados financeiros de empresas que passaram por dificuldades financeiras no passado, ajudando a prever quais empresas estão mais propensas a enfrentar problemas.